## This notebook show how to do the XAI with the trained model with TDvsCPu - ResNet 
### Load the data first

In [1]:
from utils.utils import gait_dataloader
from utils.utils import init_model,  patients_idx_to_cycles_idx, find_cycles_idx_by_patient_idx
from sklearn.model_selection import train_test_split

from utils.utils import init_model,accuracy_end_to_end, GridSearchCV

import numpy as np

from sklearn.svm import SVC

from sklearn.metrics import f1_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso
from captum.attr._core.lime import get_exp_kernel_similarity_function

def load_dataset_v1(dir_dataset,channel_first=True,flatten=True,shuffle=False,random_state=0):
    index = [0,1,2,3,4,5,6,7,8,9,14,15,16,17,18,19,20,21,22,23,24,29]
    
    npzfile = np.load(dir_dataset + 'RightPC.npz')
    
    x_train_all = npzfile['Input']
    y_train = npzfile['Output']
    y_train = np.ravel(y_train)
    x_train = x_train_all[:,:,index]
    if channel_first:
        x_train = x_train.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'LeftPC.npz')
    x_val_all = npzfile['Input']
    y_val = npzfile['Output']
    y_val = np.ravel(y_val)
    x_val = x_val_all[:,:,index]
    if channel_first:
        x_val = x_val.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'TD.npz')
    x_test_all = npzfile['Input']
    y_test = npzfile['Output']
    y_test = np.ravel(y_test)
    x_test = x_test_all[:,:,index]
    if channel_first:
        x_test = x_test.transpose(0,2,1)
    
    if flatten == True:
        x_train = x_train.reshape([x_train.shape[0],-1])
        x_val = x_val.reshape([x_val.shape[0],-1])
        x_test = x_test.reshape([x_test.shape[0],-1])
    
    nb_classes = len(np.unique(np.concatenate((y_train, y_val, y_test), axis=0)))
    
    return x_train, y_train ,x_val, y_val, x_test, y_test, nb_classes

dataset = "archives/AQM/TDvsPC_new_no_shuffle/"

x_R, y_R ,x_L , y_L, x_TD, y_TD, nb_classes = load_dataset_v1(dataset,channel_first=True,flatten=False,shuffle=False,random_state=0)

X_d = np.concatenate([x_R,x_L,x_TD])
# X_d = X_d.reshape([len(X_d),-1])
y_d = np.concatenate([y_R,y_L,y_TD])
import  pandas as pd

R_data = pd.read_csv('DataPC - Right.csv', header=None)
R_idx = R_data[2].to_numpy()


L_data = pd.read_csv('DataPC - Left.csv', header=None)
L_idx = L_data[2].to_numpy()
L_idx = L_idx + R_idx[-1]

TD_data = pd.read_csv('DataTD - By Subject.csv', header=None)
TD_idx = TD_data[2].to_numpy()
TD_idx = TD_idx + L_idx[-1]

cycle_end_idx = np.concatenate([R_idx,L_idx,TD_idx])
cycle_end_idx

patient_index_range = np.arange(len(R_idx)+len(L_idx)+len(TD_idx))
patient_class = np.concatenate([np.ones(len(R_idx)+len(L_idx)), np.zeros(len(TD_idx))])

x_train, x_test, y_train, y_test = train_test_split(patient_index_range, patient_class, test_size=0.4, stratify=patient_class,random_state=0)
x_val, x_test, y_val, y_test = train_test_split(x_test,y_test, test_size=0.75, random_state=0, stratify=y_test)
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))

/home/zylan/anaconda3/envs/env_gait/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_4524/4063324783.py:68: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  R_data = pd.read_csv('DataPC - Right.csv', header=None)


(array([0., 1.]), array([35, 89]))
(array([0., 1.]), array([ 6, 14]))
(array([0., 1.]), array([18, 45]))


/tmp/ipykernel_4524/4063324783.py:72: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  L_data = pd.read_csv('DataPC - Left.csv', header=None)


In [2]:
cycle_end_idx

array([  10,   34,   63,   72,   89,  100,  123,  146,  166,  176,  188,
        209,  236,  259,  273,  281,  299,  308,  327,  347,  370,  387,
        415,  441,  457,  482,  502,  521,  534,  539,  563,  579,  596,
        619,  633,  646,  662,  674,  695,  709,  726,  739,  752,  769,
        790,  815,  832,  847,  870,  890,  908,  929,  963,  987, 1007,
       1034, 1056, 1071, 1090, 1119, 1134, 1161, 1193, 1210, 1232, 1253,
       1271, 1286, 1312, 1321, 1333, 1358, 1379, 1400, 1416, 1439, 1461,
       1491, 1512, 1528, 1539, 1549, 1565, 1581, 1602, 1624, 1649, 1669,
       1683, 1705, 1714, 1736, 1752, 1767, 1781, 1796, 1812, 1826, 1844,
       1849, 1875, 1888, 1908, 1928, 1940, 1966, 1973, 1982, 1998, 2017,
       2033, 2041, 2055, 2071, 2091, 2100, 2111, 2120, 2128, 2141, 2157,
       2167, 2193, 2210, 2233, 2246, 2270, 2288, 2306, 2348, 2361, 2372,
       2388, 2422, 2441, 2445, 2470, 2491, 2511, 2543, 2574, 2605, 2631,
       2646, 2671, 2701, 2717, 2729, 2738, 2740, 27

In [3]:
from utils.utils import gait_dataloader
from utils.utils import init_model
## lime
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
class Dataset_torch(Dataset):

    def __init__(self, data,with_label=True):
        self.with_label =  with_label
        
        if self.with_label:
            self.data_x, self.data_y = data
        else:
            self.data_x = data
    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        if self.with_label:
            return self.data_x[idx], self.data_y[idx]
        else:
            return self.data_x[idx]

### Load the model with best accuracy

In [4]:
classifer = init_model(model_name='ResNet').load('TDvsCPu/new_train_lrR_ET_TDvsCPu_full_5_original_weight_ResNet/model-epoch=04-val_loss=0.96-val_accuracy=0.92.ckpt')

/home/zylan/Desktop/paper2/classifiers/DL_classifier.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_dir,map_location=torch.device('cpu'))

## Compute the XAI of IntegratedGradients and save the results

In [6]:
from captum.attr import IntegratedGradients
target=0
ig = IntegratedGradients(classifer.model)
inputs = torch.from_numpy(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]==target]).to(torch.float32)
attrs_0 = ig.attribute(inputs,target=target)

from captum.attr import IntegratedGradients
target=1
ig = IntegratedGradients(classifer.model)
inputs = torch.from_numpy(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]==target]).to(torch.float32)
attrs_1 = ig.attribute(inputs,target=target)

np.save("./XAI_value/ig_tdvscp_weight_ResNet_0", attrs_0)
np.save("./XAI_value/ig_tdvscp_weight_ResNet_1", attrs_1)

## Compute the XAI of LIME and save the results

In [5]:
exp_eucl_distance = get_exp_kernel_similarity_function('euclidean', kernel_width=100)

lr_lime = Lime(
    classifer.model, 
    interpretable_model=SkLearnLinearRegression(),  # build-in wrapped sklearn Linear Regression
    similarity_func=exp_eucl_distance
)

target = 0

attrs_0 = lr_lime.attribute(
    torch.from_numpy(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]==target]).to(torch.float32),
    target=target,
    n_samples=5000,
    perturbations_per_eval=200,
    show_progress=True
).squeeze(0)

target = 1

attrs_1 = lr_lime.attribute(
    torch.from_numpy(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]==target]).to(torch.float32),
    target=target,
    n_samples=5000,
    perturbations_per_eval=200,
    show_progress=True
).squeeze(0)

np.save("./XAI_value/lime_tdvscp_weight_ResNet_0", attrs_0)
np.save("./XAI_value/lime_tdvscp_weight_ResNet_1", attrs_1)

/home/zhengyang/Desktop/Git_resp_limesegment/classifiers/ResNet.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_layer = self.softmax(output_layer)
/home/zhengyang/anaconda3/envs/env_gait/lib/python3.9/site-packages/captum/attr/_core/lime.py:1115: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.
  warnings.warn(


Lime attribution:   0%|          | 0/25 [00:00<?, ?it/s]

Lime attribution:   0%|          | 0/25 [00:00<?, ?it/s]

/home/zhengyang/Desktop/Git_resp_limesegment/classifiers/ResNet.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_layer = self.softmax(output_layer)


Lime attribution:   0%|          | 0/25 [00:00<?, ?it/s]

...

/home/zhengyang/Desktop/Git_resp_limesegment/classifiers/ResNet.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_layer = self.softmax(output_layer)


Lime attribution:   0%|          | 0/25 [00:00<?, ?it/s]

/home/zhengyang/Desktop/Git_resp_limesegment/classifiers/ResNet.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_layer = self.softmax(output_layer)


Lime attribution:   0%|          | 0/25 [00:00<?, ?it/s]

/home/zhengyang/Desktop/Git_resp_limesegment/classifiers/ResNet.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_layer = self.softmax(output_layer)


## Compute the XAI of DeepLift and save the results
### Due to memory limitations, calculations are performed in segments.

In [ ]:
from captum.attr import DeepLiftShap
import math

target=0

inputs = torch.from_numpy(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]==target]).to(torch.float32)

inputs = torch.split(inputs, split_size_or_sections=math.ceil(len(inputs)/20), dim=0)


attrs_0_all = []

for i in inputs:
    
    dls = DeepLiftShap(classifer.model)
    i.requires_grad=True
    attrs_0 = dls.attribute(i,target=target,baselines=i*0)
    attrs_0_all.append(attrs_0.detach().numpy())

target=1

inputs = torch.from_numpy(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]==target]).to(torch.float32)

inputs = torch.split(inputs, split_size_or_sections=math.ceil(len(inputs)/20), dim=0)


attrs_1_all = []

for i in inputs:
    
    dls = DeepLiftShap(classifer.model)
    i.requires_grad=True
    attrs_1 = dls.attribute(i,target=target,baselines=i*0)
    attrs_1_all.append(attrs_1.detach().numpy())

np.save("./XAI_value/DeepLiftShap_tdvscp_ResNet_0", np.concatenate(attrs_0_all))
np.save("./XAI_value/DeepLiftShap_tdvscp_ResNet_1", np.concatenate(attrs_1_all))